In [3]:
from keras.applications import VGG16

# Setting the input size now to 64 x 64 pixel 
img_rows = 64
img_cols = 64

# Re-loads the VGG16 model without the top or FC layers
vgg16 = VGG16(weights = 'imagenet', 
                 include_top = False , 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in vgg16.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(vgg16.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [4]:
def addTopModel(bottom_model, num_classes, D=256):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [5]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = './train'
validation_data_dir = './valid'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

# Re-loads the VGG16 model without the top or FC layers
vgg16 = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Freeze layers
for layer in vgg16.layers:
    layer.trainable = False
    
# Number of classes in the Flowers-17 dataset
num_classes = 4

FC_Head = addTopModel(vgg16, num_classes)

model = Model(inputs=vgg16.input, outputs=FC_Head)

print(model.summary())

Found 2876 images belonging to 4 classes.
Found 719 images belonging to 4 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
___________________________________________________________

In [43]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
                   
checkpoint = ModelCheckpoint("Trained_Models/test2_1.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 3,
                              verbose = 1,
                              min_delta = 0.00001)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint, reduce_lr]

# Note we use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.0001),
              metrics = ['accuracy'])


history = model.fit_generator(
    train_generator,
    steps_per_epoch = 2876 ,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = 719)

model.save("Trained_Models/test2_1.h5")

Epoch 1/25
2876/2876 [==============================] - 2037s 708ms/step - loss: 0.4938 - acc: 0.7919 - val_loss: 0.3817 - val_acc: 0.8329

Epoch 00001: val_loss improved from inf to 0.38168, saving model to Trained_Models/test2_1.h5
Epoch 2/25
2876/2876 [==============================] - 2013s 700ms/step - loss: 0.3990 - acc: 0.8328 - val_loss: 0.3435 - val_acc: 0.8482

Epoch 00002: val_loss improved from 0.38168 to 0.34350, saving model to Trained_Models/test2_1.h5
Epoch 3/25
2876/2876 [==============================] - 2029s 705ms/step - loss: 0.3577 - acc: 0.8501 - val_loss: 0.3805 - val_acc: 0.8315

Epoch 00003: val_loss did not improve from 0.34350
Epoch 4/25
2876/2876 [==============================] - 2017s 701ms/step - loss: 0.3362 - acc: 0.8607 - val_loss: 0.3066 - val_acc: 0.8774

Epoch 00004: val_loss improved from 0.34350 to 0.30661, saving model to Trained_Models/test2_1.h5
Epoch 5/25
2876/2876 [==============================] - 64010s 22s/step - loss: 0.3193 - acc: 0.867

In [7]:

nb_train_samples = 2876  
nb_validation_samples = 719 
epochs = 25
batch_size = 32
import matplotlib.pyplot as plt
from keras.models import load_model
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
classifier = load_model('Trained_Models/test2_1.h5')
# We need to recreate our validation generator with shuffle = false

valiation_genrator=validation_datagen.flow_from_directory(validation_data_dir,
                                                          target_size=(img_rows,img_cols),
                                                          batch_size=batch_size,
                                                          class_mode='categorical',
                                                         shuffle=False)

class_labels = valiation_genrator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())


#Confution Matrix and Classification Report
Y_pred = classifier.predict_generator(valiation_genrator,  nb_validation_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)


Found 719 images belonging to 4 classes.


In [45]:
 y_pred

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 2, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 3, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 2, 0, 0, 0, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,

In [11]:
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(valiation_genrator.classes, y_pred, target_names=target_names))
accuracy_score(valiation_genrator.classes,y_pred)

Classification Report
                                            precision    recall  f1-score   support

                         Grape___Black_rot       0.87      0.77      0.82       235
              Grape___Esca_(Black_Measles)       0.80      0.85      0.83       200
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)       0.96      0.98      0.97       200
                           Grape___healthy       0.91      0.98      0.94        84

                                 micro avg       0.88      0.88      0.88       719
                                 macro avg       0.88      0.90      0.89       719
                              weighted avg       0.88      0.88      0.88       719



0.8776077885952712

# for single test image


In [34]:
import cv2
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing import image # Need to reload as opencv2 seems to have a conflict
# create a list containing the class labels
img = image.load_img('./teste data/a/h.jpg', target_size=(64, 64))

x = image.img_to_array(img)
x = x * 1./255
x = np.expand_dims(x, axis=0)
images = np.vstack([x]) 
    
    # Get Prediction
res = np.argmax(classifier.predict(images))
disease_list=['Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy']    
    # Show image with predicted class
#draw_test("Prediction", res, input_original)     
# preds_vgg_model = classifier.predict(x)
disease = disease_list[int(res)]
disease



 

    
    
    

# print(preds_vgg_model)
# a=list(preds_vgg_model[0])

# maxpos = a.index(max(a)) 
# print(maxpos)

'Grape___healthy'

In [30]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

DD=list(train_generator.class_indices.keys())
def draw_test(name, pred, im):

    rsc = DD[int(pred)]
    print(rsc)
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 150, 0, 0, 200 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, rsc, (10, 20) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path,i):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
 
    path_class = folders[i]
    i=i+1
    
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    
cnt=0
for i in range(5):
    input_im = getRandomImage("./teste data/",cnt)
    #input_im = image.load_img('./teste data/black_rot/3.jpg', target_size=(64, 64))
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (64, 64), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,64,64,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im))
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
Grape___Black_rot
Grape___healthy
Grape___healthy
Grape___Black_rot
